# Wines Points prediction 

Submission Date : 3.6.2023
Task: Predict the wine score given the inputs
Instructions:
 * Use logistic regression as benchmark model
 * Use sklearn pipeliens + cv + grid search with sklearn models (e.g. KNNs, RandomForest, etc.)
 * Compare all models on proper metric (your choice)

For DNN course project:
* Use sklearn pipeliens with tensorflow models (w/wo embeddings, LSTMs, RNNs, Transformers etc.)
* Compare all models on proper metric (your choice)

In [9]:
%load_ext autoreload
%autoreload 2
import sys; sys.path.append('../')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Here we will try to predict the points a wine will get based on known characteristics (i.e. features, in the ML terminology). The mine point in this stage is to establish a simple, ideally super cost effective, basline.
In the real world there is a tradeoff between complexity and perforamnce, and the DS job, among others, is to present a tradeoff tables of what performance is achivalbel at what complexity level. 

to which models with increased complexity and resource demands will be compared. Complexity should then be translated into cost. For example:
 * Compute cost 
 * Maintenance cost
 * Serving costs (i.e. is new platform needed?) 
 

## Loading the data

In [2]:
import pandas as pd
import cufflinks as cf; cf.go_offline()

In [3]:
wine_reviews = pd.read_csv("C:/Users/CBS/Python_ML_2023/data/winemag-data-130k-v2.csv")
wine_reviews.shape

(129971, 14)

In [4]:
import numpy as np
import seaborn as sns # visualization
import matplotlib.pyplot as plt # visualization
import missingno as msno # visualizatin for missing values

#import warnings
#warnings.filterwarnings("ignore") # ignore warnings

In [5]:
df=wine_reviews.copy()

In [6]:
df.head(1)

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 129971 entries, 0 to 129970
Data columns (total 14 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   Unnamed: 0             129971 non-null  int64  
 1   country                129908 non-null  object 
 2   description            129971 non-null  object 
 3   designation            92506 non-null   object 
 4   points                 129971 non-null  int64  
 5   price                  120975 non-null  float64
 6   province               129908 non-null  object 
 7   region_1               108724 non-null  object 
 8   region_2               50511 non-null   object 
 9   taster_name            103727 non-null  object 
 10  taster_twitter_handle  98758 non-null   object 
 11  title                  129971 non-null  object 
 12  variety                129970 non-null  object 
 13  winery                 129971 non-null  object 
dtypes: float64(1), int64(2), object(11)


In [8]:
df.columns

Index(['Unnamed: 0', 'country', 'description', 'designation', 'points',
       'price', 'province', 'region_1', 'region_2', 'taster_name',
       'taster_twitter_handle', 'title', 'variety', 'winery'],
      dtype='object')

In [9]:
df.drop('Unnamed: 0', axis=1, inplace=True)

In [10]:
df.columns

Index(['country', 'description', 'designation', 'points', 'price', 'province',
       'region_1', 'region_2', 'taster_name', 'taster_twitter_handle', 'title',
       'variety', 'winery'],
      dtype='object')

In [11]:
round(df.describe(),3)

,points,price
count,129971.000,120975.000
mean,88.447,35.363
std,3.040,41.022
min,80.000,4.000
25%,86.000,17.000
50%,88.000,25.000
75%,91.000,42.000
max,100.000,3300.000


# Missing Value Analysis

In [12]:
df.isnull().sum()

country                     63
description                  0
designation              37465
points                       0
price                     8996
province                    63
region_1                 21247
region_2                 79460
taster_name              26244
taster_twitter_handle    31213
title                        0
variety                      1
winery                       0
dtype: int64

In [13]:
# Missing Value Table
def missing_value_table(df):
    missing_value = df.isna().sum().sort_values(ascending=False)
    missing_value_percent = 100 * df.isna().sum()//len(df)
    missing_value_table = pd.concat([missing_value, missing_value_percent], axis=1)
    missing_value_table_return = missing_value_table.rename(columns = {0 : 'Missing Values', 1 : '% Value'})
    cm = sns.light_palette("blue", as_cmap=True)
    missing_value_table_return = missing_value_table_return.style.background_gradient(cmap=cm)
    return missing_value_table_return
  
missing_value_table(df)

,Missing Values,% Value
region_2,79460,61
designation,37465,28
taster_twitter_handle,31213,24
taster_name,26244,20
region_1,21247,16
price,8996,6
country,63,0
province,63,0
variety,1,0
description,0,0


In [14]:
df.drop("region_2", axis = 1, inplace = True)

In [15]:
 df['price'] = df['price'].fillna(df.groupby(['country','province'])['price'].transform('mean'))

In [16]:
# Missing Value Table
def missing_value_table(df):
    missing_value = df.isna().sum().sort_values(ascending=False)
    missing_value_percent = 100 * df.isna().sum()//len(df)
    missing_value_table = pd.concat([missing_value, missing_value_percent], axis=1)
    missing_value_table_return = missing_value_table.rename(columns = {0 : 'Missing Values', 1 : '% Value'})
    cm = sns.light_palette("blue", as_cmap=True)
    missing_value_table_return = missing_value_table_return.style.background_gradient(cmap=cm)
    return missing_value_table_return
  
missing_value_table(df)

,Missing Values,% Value
designation,37465,28
taster_twitter_handle,31213,24
taster_name,26244,20
region_1,21247,16
country,63,0
province,63,0
price,7,0
variety,1,0
description,0,0
points,0,0


In [17]:
df = df[~(df.price.isnull())]

In [18]:
# Missing Value Table
def missing_value_table(df):
    missing_value = df.isna().sum().sort_values(ascending=False)
    missing_value_percent = 100 * df.isna().sum()//len(df)
    missing_value_table = pd.concat([missing_value, missing_value_percent], axis=1)
    missing_value_table_return = missing_value_table.rename(columns = {0 : 'Missing Values', 1 : '% Value'})
    cm = sns.light_palette("blue", as_cmap=True)
    missing_value_table_return = missing_value_table_return.style.background_gradient(cmap=cm)
    return missing_value_table_return
  
missing_value_table(df)

,Missing Values,% Value
designation,37465,28
taster_twitter_handle,31213,24
taster_name,26244,20
region_1,21240,16
country,59,0
province,59,0
variety,1,0
description,0,0
points,0,0
price,0,0


In [19]:
df.fillna(value = {'designation': 'other'}, inplace = True)

In [20]:
def missing_value_table(df):
    missing_value = df.isna().sum().sort_values(ascending=False)
    missing_value_percent = 100 * df.isna().sum()//len(df)
    missing_value_table = pd.concat([missing_value, missing_value_percent], axis=1)
    missing_value_table_return = missing_value_table.rename(columns = {0 : 'Missing Values', 1 : '% Value'})
    cm = sns.light_palette("blue", as_cmap=True)
    missing_value_table_return = missing_value_table_return.style.background_gradient(cmap=cm)
    return missing_value_table_return
  
missing_value_table(df)

,Missing Values,% Value
taster_twitter_handle,31213,24
taster_name,26244,20
region_1,21240,16
country,59,0
province,59,0
variety,1,0
description,0,0
designation,0,0
points,0,0
price,0,0


In [21]:
df.fillna(value = {'region_1': 'other','country': 'other','province': 'other'}, inplace = True)

In [22]:
def missing_value_table(df):
    missing_value = df.isna().sum().sort_values(ascending=False)
    missing_value_percent = 100 * df.isna().sum()//len(df)
    missing_value_table = pd.concat([missing_value, missing_value_percent], axis=1)
    missing_value_table_return = missing_value_table.rename(columns = {0 : 'Missing Values', 1 : '% Value'})
    cm = sns.light_palette("blue", as_cmap=True)
    missing_value_table_return = missing_value_table_return.style.background_gradient(cmap=cm)
    return missing_value_table_return
  
missing_value_table(df)

,Missing Values,% Value
taster_twitter_handle,31213,24
taster_name,26244,20
variety,1,0
country,0,0
description,0,0
designation,0,0
points,0,0
price,0,0
province,0,0
region_1,0,0


In [23]:
df.fillna(value = {'variety': 'other'}, inplace = True)

In [24]:
def missing_value_table(df):
    missing_value = df.isna().sum().sort_values(ascending=False)
    missing_value_percent = 100 * df.isna().sum()//len(df)
    missing_value_table = pd.concat([missing_value, missing_value_percent], axis=1)
    missing_value_table_return = missing_value_table.rename(columns = {0 : 'Missing Values', 1 : '% Value'})
    cm = sns.light_palette("blue", as_cmap=True)
    missing_value_table_return = missing_value_table_return.style.background_gradient(cmap=cm)
    return missing_value_table_return
  
missing_value_table(df)

,Missing Values,% Value
taster_twitter_handle,31213,24
taster_name,26244,20
country,0,0
description,0,0
designation,0,0
points,0,0
price,0,0
province,0,0
region_1,0,0
title,0,0


In [25]:
df.fillna(value = {'taster_name': 'other'}, inplace = True)

In [26]:
df.taster_name.value_counts()

other                 26244
Roger Voss            25512
Michael Schachner     15134
Kerin O’Keefe         10776
Virginie Boone         9537
Paul Gregutt           9532
Matt Kettmann          6332
Joe Czerwinski         5147
Sean P. Sullivan       4966
Anna Lee C. Iijima     4415
Jim Gordon             4177
Anne Krebiehl MW       3685
Lauren Buzzeo          1831
Susan Kostrzewa        1084
Mike DeSimone           514
Jeff Jenssen            491
Alexander Peartree      415
Carrie Dykes            139
Fiona Adams              27
Christina Pickard         6
Name: taster_name, dtype: int64

In [27]:
df.groupby('taster_name')['points'].mean().sort_values(ascending=False)

taster_name
Anne Krebiehl MW      90.562551
Matt Kettmann         90.008686
Virginie Boone        89.213379
Mike DeSimone         89.101167
Paul Gregutt          89.082564
Kerin O’Keefe         88.867947
Sean P. Sullivan      88.755739
Roger Voss            88.708490
Jim Gordon            88.626287
Joe Czerwinski        88.536235
Anna Lee C. Iijima    88.415629
Jeff Jenssen          88.319756
Christina Pickard     87.833333
other                 87.795115
Lauren Buzzeo         87.733479
Michael Schachner     86.907493
Fiona Adams           86.888889
Susan Kostrzewa       86.611624
Carrie Dykes          86.395683
Alexander Peartree    85.855422
Name: points, dtype: float64

In [28]:
df.columns

Index(['country', 'description', 'designation', 'points', 'price', 'province',
       'region_1', 'taster_name', 'taster_twitter_handle', 'title', 'variety',
       'winery'],
      dtype='object')

In [29]:
import numpy as np

# Define the conditions
conditions = [
    (df['taster_name'] == 'Anne Krebiehl MW') | (df['taster_name'] == 'Matt Kettmann')
    | (df['taster_name'] == 'Virginie Boone')
     | (df['taster_name'] == 'Mike DeSimone')   | (df['taster_name'] == 'Paul Gregutt') 
    ,
    (df['taster_name'] == 'Kerin O’Keefe') | (df['taster_name'] == 'Sean P. Sullivan')
     | (df['taster_name'] == 'Roger Voss')  | (df['taster_name'] == 'Jim Gordon')
     | (df['taster_name'] == 'Joe Czerwinski') | (df['taster_name'] == 'Anna Lee C. Iijima')
    | (df['taster_name'] == 'Jeff Jenssen')  | (df['taster_name'] == 'other')
    ,
   ( df['taster_name'] == 'Christina Pickard')   | (df['taster_name'] == 'Lauren Buzzeo')
   | ( df['taster_name'] == 'Michael Schachner')  | ( df['taster_name'] == 'Fiona Adams ')
    | ( df['taster_name'] == 'Susan Kostrzewa ')   | ( df['taster_name'] == 'Carrie Dykes ')
     | ( df['taster_name'] == 'Alexander Peartree ')
]

# Define the corresponding categories
categories = ['Category A', 'Category B', 'Category C']

# Create the new column using np.select
df['taster_weight_cat'] = np.select(conditions, categories,default='Other')

# Print the DataFrame



In [30]:
df['taster_weight_cat'].value_counts()

Category B    81728
Category A    25915
Category C    16971
Other          5350
Name: taster_weight_cat, dtype: int64

In [31]:
def missing_value_table(df):
    missing_value = df.isna().sum().sort_values(ascending=False)
    missing_value_percent = 100 * df.isna().sum()//len(df)
    missing_value_table = pd.concat([missing_value, missing_value_percent], axis=1)
    missing_value_table_return = missing_value_table.rename(columns = {0 : 'Missing Values', 1 : '% Value'})
    cm = sns.light_palette("blue", as_cmap=True)
    missing_value_table_return = missing_value_table_return.style.background_gradient(cmap=cm)
    return missing_value_table_return
  
missing_value_table(df)

,Missing Values,% Value
taster_twitter_handle,31213,24
country,0,0
description,0,0
designation,0,0
points,0,0
price,0,0
province,0,0
region_1,0,0
taster_name,0,0
title,0,0


In [32]:
df.taster_twitter_handle.head(20)

0      @kerinokeefe
1        @vossroger
2       @paulgwine 
3               NaN
4       @paulgwine 
5       @wineschach
6      @kerinokeefe
7        @vossroger
8               NaN
9        @vossroger
10          @vboone
11       @vossroger
12          @vboone
13     @kerinokeefe
14    @mattkettmann
15              NaN
16      @wineschach
17      @wineschach
18      @wineschach
19              NaN
Name: taster_twitter_handle, dtype: object

In [33]:
df.fillna(value = {'taster_twitter_handle': 'other'}, inplace = True)

In [34]:
def missing_value_table(df):
    missing_value = df.isna().sum().sort_values(ascending=False)
    missing_value_percent = 100 * df.isna().sum()//len(df)
    missing_value_table = pd.concat([missing_value, missing_value_percent], axis=1)
    missing_value_table_return = missing_value_table.rename(columns = {0 : 'Missing Values', 1 : '% Value'})
    cm = sns.light_palette("blue", as_cmap=True)
    missing_value_table_return = missing_value_table_return.style.background_gradient(cmap=cm)
    return missing_value_table_return
  
missing_value_table(df)

,Missing Values,% Value
country,0,0
description,0,0
designation,0,0
points,0,0
price,0,0
province,0,0
region_1,0,0
taster_name,0,0
taster_twitter_handle,0,0
title,0,0


In [35]:
df.drop('taster_twitter_handle', axis=1, inplace=True)

In [36]:
df.columns


Index(['country', 'description', 'designation', 'points', 'price', 'province',
       'region_1', 'taster_name', 'title', 'variety', 'winery',
       'taster_weight_cat'],
      dtype='object')

In [37]:
df.description

0         Aromas include tropical fruit, broom, brimston...
1         This is ripe and fruity, a wine that is smooth...
2         Tart and snappy, the flavors of lime flesh and...
3         Pineapple rind, lemon pith and orange blossom ...
4         Much like the regular bottling from 2012, this...
                                ...                        
129966    Notes of honeysuckle and cantaloupe sweeten th...
129967    Citation is given as much as a decade of bottl...
129968    Well-drained gravel soil gives this wine its c...
129969    A dry style of Pinot Gris, this is crisp with ...
129970    Big, rich and off-dry, this is powered by inte...
Name: description, Length: 129964, dtype: object

In [38]:
df = df.reset_index(drop=True)


In [39]:
len(df)

129964

# cleaning the text

In [40]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')
stop_words = stopwords.words('english')

print(stop_words)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\CBS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [41]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
corpus= []
all_stopwords=stopwords.words('english')
words_to_remove = ["not", "doesn't", "won't", "wouldn", "wouldn't", "isn't", "shouldn't",
                   "wasn't", "weren't", "doesn't", "don't", "couldn't", "didn", "didn't",
                   "doesn", "doesn't"]
for word in words_to_remove:
    if word in all_stopwords:
        all_stopwords.remove(word)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\CBS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [48]:
print(all_stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'only', 'own', 'same', 'so', 'than', 'too', '

In [46]:
len(df)

129964

In [41]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
corpus= []
for i in range (0, 129964):
    review =re.sub('[^a-zA-Z]' , ' ' , df['description'][i])
    review=review.lower()
    review=review.split()
    ps=PorterStemmer()
    all_stopwords=stopwords.words('english')
    all_stopwords.remove('not')
    review=[ ps.stem(word) for word in review if not word in set(all_stopwords)]
    review= ' '.join(review)
    corpus.append(review)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\CBS\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [42]:
len(corpus)

129964

In [43]:
df['clean_text'] = corpus

In [44]:
df.head(5)

,country,description,designation,points,price,province,region_1,taster_name,title,variety,winery,taster_weight_cat,clean_text
0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,27.623477,Sicily & Sardinia,Etna,Kerin O’Keefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,Category B,aroma includ tropic fruit broom brimston dri h...
1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.000000,Douro,other,Roger Voss,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,Category B,ripe fruiti wine smooth still structur firm ta...
2,US,"Tart and snappy, the flavors of lime flesh and...",other,87,14.000000,Oregon,Willamette Valley,Paul Gregutt,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,Category A,tart snappi flavor lime flesh rind domin green...
3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.000000,Michigan,Lake Michigan Shore,Alexander Peartree,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,Other,pineappl rind lemon pith orang blossom start a...
4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.000000,Oregon,Willamette Valley,Paul Gregutt,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,Category A,much like regular bottl come across rather rou...


In [45]:
import re

pattern = r'\((.*?)\)'
words = []
for i in range(0, 129964):
    review = df['title'][i]
    extracted_words = re.findall(pattern, review)
    cleaned_words = [word.strip('()') for word in extracted_words]
    words.append(cleaned_words)
    
df['extracted_words'] = words

In [46]:
df['extracted_words'].value_counts()>1

[]                                          True
[Napa Valley]                               True
[Columbia Valley (WA]                       True
[Russian River Valley]                      True
[California]                                True
                                           ...  
[Côtes du Roussillon Villages Caramany]    False
[Hrvatsko Primorje]                        False
[Grave del Friuli]                         False
[Mâcon-Prissé]                             False
[Paestum]                                  False
Name: extracted_words, Length: 1572, dtype: bool

In [47]:
word_counts = df['extracted_words'].value_counts()
filtered_word_counts = word_counts[word_counts > 1]


In [48]:
filtered_word_counts

[]                          10518
[Napa Valley]                4478
[Columbia Valley (WA]        4124
[Russian River Valley]       3091
[California]                 2629
                            ...  
[sic, Willamette Valley]        2
[Santa Catarina]                2
[Mâcon-Uchizy]                  2
[Armenia]                       2
[Napa-Carneros]                 2
Name: extracted_words, Length: 1348, dtype: int64

In [51]:
df['extracted_digits'].value_counts()

2013    15732
2012    15622
2014    15439
2011    12468
2010    12084
        ...  
735         1
1607        1
813         1
1621        1
733         1
Name: extracted_digits, Length: 194, dtype: int64

In [50]:
import re
pattern = r'\d+'
digits = []
for i in range(0, 129964):
    review = df['title'][i]
    extracted_digits = re.findall(pattern, review)
    if extracted_digits:
        first_four_digits = extracted_digits[0]
    else:
        first_four_digits = None
    digits.append(first_four_digits)

df['extracted_digits'] = digits



In [52]:
df.columns

Index(['country', 'description', 'designation', 'points', 'price', 'province',
       'region_1', 'taster_name', 'title', 'variety', 'winery',
       'taster_weight_cat', 'clean_text', 'extracted_words',
       'extracted_digits'],
      dtype='object')

In [53]:
df['extracted_digits'] = df['extracted_digits'].fillna(0)


In [54]:
df['extracted_digits'].value_counts().head()


2013    15732
2012    15622
2014    15439
2011    12468
2010    12084
Name: extracted_digits, dtype: int64

In [55]:
df['extracted_digits'] = df['extracted_digits'].astype(int)
df['extracted_digits'].sort_values(ascending=True).head(100)


35049     0
127803    0
4405      0
4398      0
121462    0
         ..
69903     0
98773     0
24890     0
122639    0
24888     0
Name: extracted_digits, Length: 100, dtype: int32

In [56]:
df['extracted_digits']

0         2013
1         2011
2         2013
3         2013
4         2012
          ... 
129959    2013
129960    2004
129961    2013
129962    2012
129963    2012
Name: extracted_digits, Length: 129964, dtype: int32

In [57]:
df['extracted_digits'] = df['extracted_digits'].apply(lambda digits: str(digits) if len(str(digits)) == 4 else 'none')


In [58]:
df['extracted_digits']

0         2013
1         2011
2         2013
3         2013
4         2012
          ... 
129959    2013
129960    2004
129961    2013
129962    2012
129963    2012
Name: extracted_digits, Length: 129964, dtype: object

In [59]:
df['extracted_digits']=='none'

0         False
1         False
2         False
3         False
4         False
          ...  
129959    False
129960    False
129961    False
129962    False
129963    False
Name: extracted_digits, Length: 129964, dtype: bool

In [60]:
count_none = df['extracted_digits'].value_counts()['none']
print(count_none)


5331


In [61]:
unique_values = df['extracted_digits'].unique()
print(unique_values)


['2013' '2011' '2012' '2010' '2007' '2009' '2008' '2014' '2015' 'none'
 '2016' '2004' '2003' '2006' '2001' '2005' '2002' '1887' '2000' '1999'
 '1991' '1997' '1996' '1877' '2017' '1995' '1872' '1637' '1868' '1898'
 '1492' '1998' '7200' '1852' '1994' '1992' '1929' '1912' '1875' '1976'
 '1964' '1848' '1870' '1856' '1983' '1967' '1990' '1988' '1827' '1860'
 '1850' '1000' '1980' '1987' '1989' '1993' '1969' '1882' '1935' '1503'
 '1821' '1973' '1978' '1965' '1968' '1947' '1963' '1070' '1985' '1927'
 '1904' '1847' '1982' '1986' '1752' '1789' '1607' '1621' '1919' '1957'
 '1966' '1984' '1961' '1845' '1952' '1150' '1941' '1974' '3000' '1934'
 '1945']


In [62]:
df['extracted_digits'] = df['extracted_digits'].replace(['3000', '1000', '1070','1150'], "none")


In [63]:
unique_values = df['extracted_digits'].unique()
print(unique_values)


['2013' '2011' '2012' '2010' '2007' '2009' '2008' '2014' '2015' 'none'
 '2016' '2004' '2003' '2006' '2001' '2005' '2002' '1887' '2000' '1999'
 '1991' '1997' '1996' '1877' '2017' '1995' '1872' '1637' '1868' '1898'
 '1492' '1998' '7200' '1852' '1994' '1992' '1929' '1912' '1875' '1976'
 '1964' '1848' '1870' '1856' '1983' '1967' '1990' '1988' '1827' '1860'
 '1850' '1980' '1987' '1989' '1993' '1969' '1882' '1935' '1503' '1821'
 '1973' '1978' '1965' '1968' '1947' '1963' '1985' '1927' '1904' '1847'
 '1982' '1986' '1752' '1789' '1607' '1621' '1919' '1957' '1966' '1984'
 '1961' '1845' '1952' '1941' '1974' '1934' '1945']


In [64]:
counts = df['extracted_digits'].value_counts()
filtered_counts = counts[counts >3 ]


In [65]:
df.columns

Index(['country', 'description', 'designation', 'points', 'price', 'province',
       'region_1', 'taster_name', 'title', 'variety', 'winery',
       'taster_weight_cat', 'clean_text', 'extracted_words',
       'extracted_digits'],
      dtype='object')

In [66]:
df.to_csv('df_wine_clean.csv', index=False)


In [242]:
df.columns

Index(['country', 'description', 'designation', 'points', 'price', 'province',
       'region_1', 'taster_name', 'title', 'variety', 'winery',
       'taster_weight_cat', 'clean_text', 'extracted_words',
       'extracted_digits'],
      dtype='object')

In [244]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error


# Separate the features and target variable
X = df.drop(['points','extracted_digits','description','title'], axis=1)
y = df['points']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the column transformer
# Define categorical and numerical features
categorical_features = [ 'country',  'designation', 'province',
  'region_1', 'taster_name', 'variety', 'winery', 'taster_weight_cat']
numerical_features = ['price']
text_features = ['clean_text']  # Assuming 'description' is the text feature column

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features),
        ('text', CountVectorizer(max_features = 1500), text_features)
    ])

# Define the pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor())
])

# Define the parameter grid for GridSearchCV
param_grid = {
    'model__n_estimators': [100, 200, 300],
    'model__max_depth': [None, 5, 10],
    'model__min_samples_split': [2, 5, 10]
}

# Perform GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Get the best model and its MSE
best_model = grid_search.best_estimator_
best_mse = -grid_search.best_score_

print("Best Model:")
print(best_model)
print("Best MSE:", best_mse)

# Evaluate the model on the test set
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Test MSE:", mse)


KeyboardInterrupt: 

In [258]:
X = df.drop(['points','designation','extracted_words','extracted_digits','description','clean_text','title','taster_name','taster_weight_cat'], axis=1)


In [79]:
df = df.sample(frac=0.5, random_state=42)

In [ ]:
len(df)

In [1]:
df.clean_text

NameError: name 'df' is not defined

# מודל פשוט 

In [ ]:
from sklearn.linear_model import LinearRegression

regressor = LinearRegression()
regressor.fit(X_train, y_train)


In [270]:
y_pred = classifier.predict(X_test)

In [ ]:

mse = mean_squared_error(y_test, y_pred)
print("Test MSE:", mse)

In [272]:
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(-1, 1), y_test.values.reshape(-1, 1)), axis=1))


[[93 93]
 [88 87]
 [96 96]
 ...
 [97 91]
 [96 91]
 [96 83]]


In [68]:
import os

current_directory = os.getcwd()

# Reading the clean csv df wine

In [4]:
import pandas as pd

file_path = r'C:\Users\CBS\Python_ML_2023\Exercises\Final Project\df_wine_clean.csv'
df = pd.read_csv(file_path)

# Now you can work with the DataFrame 'df'


In [5]:
df.columns

Index(['country', 'description', 'designation', 'points', 'price', 'province',
       'region_1', 'taster_name', 'title', 'variety', 'winery',
       'taster_weight_cat', 'clean_text', 'extracted_words',
       'extracted_digits'],
      dtype='object')

In [73]:
len(df)

129964

In [6]:
df.nunique()

country                  43
description          119948
designation           37975
points                   21
price                   542
province                423
region_1               1230
taster_name              20
title                118833
variety                 708
winery                16756
taster_weight_cat         4
clean_text           119930
extracted_words        1572
extracted_digits         87
dtype: int64

In [ ]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import category_encoders as ce

# Load the dataset (assuming you have a CSV file named 'wine_data.csv')
wine_data =df

# Separate the features and target variable
X = wine_data.drop('title','points','description','extracted_words', axis=1)
y = wine_data['points']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the column transformer
numeric_features = X.select_dtypes(include=['float64', 'int64']).columns
target_encoding_features = ['province', 'region_1','variety','extracted_words']
onehot_encoding_features = ['taster_weight_cat','taster_name','extracted_digits','country']
hashing_trick_feature = ['designation','winery']
text_features = ['clean_text'] # Assuming 'description' is the text feature column

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('target_enc', ce.TargetEncoder(), target_encoding_features),
        ('onehot_enc', OneHotEncoder(), onehot_encoding_features),
        ('hashing_trick', HashingVectorizer(n_features=10, alternate_sign=False), hashing_trick_feature),
        ('text', HashingVectorizer(n_features=100), text_features)
    ])

# Define the pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor())
])

# Define the parameter grid for GridSearchCV
param_grid = {
    'model__n_estimators': [100, 200, 300],
    'model__max_depth': [None, 5, 10],
    'model__min_samples_split': [2, 5, 10]
}

# Perform GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Get the best model and its MSE
best_model = grid_search.best_estimator_
best_mse = -grid_search.best_score_

print("Best Model:")
print(best_model)
print("Best MSE:", best_mse)

# Evaluate the model on the test set
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Test MSE:", mse)

In [13]:

import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import category_encoders as ce

# Load the dataset (assuming you have a DataFrame named 'df' with columns 'clean_text' and 'points')
wine_data = df

# Separate the features and target variable
X = wine_data['clean_text']
y = wine_data['points']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Define the column transformer
text_features = ['clean_text']  # Assuming 'clean_text' is the text feature column
preprocessor = ColumnTransformer(
    transformers=[
        ('text', CountVectorizer(), text_features)
    ])

# Define the pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(n_estimators=100, max_depth=None, min_samples_split=2))
])




In [24]:
import sklearn

In [25]:
sklearn.__version__

'1.2.2'

In [21]:
y_train

28418     86
35956     93
85408     90
101214    85
82259     92
          ..
128106    85
103694    90
860       90
15795     94
121958    90
Name: points, Length: 103971, dtype: int64

In [22]:
# Train the model
pipeline.fit(X_train.values, y_train.values)


IndexError: tuple index out of range

In [15]:

# Evaluate the model on the test set
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Test MSE:", mse)


IndexError: tuple index out of range

# Creating the Bag of Words model

In [38]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1500)
X = cv.fit_transform(wine_data['clean_text']).toarray()
y = wine_data['points'].values

In [39]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [40]:
from sklearn.ensemble import RandomForestRegressor

# Create a Random Forest Regressor
regressor = RandomForestRegressor(n_estimators=100, max_depth=None, min_samples_split=2)

# Train the Random Forest Regressor
regressor.fit(X_train, y_train)

RandomForestRegressor()

In [41]:
# Evaluate the model on the test set
y_pred = regressor.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Test MSE:", mse)

Test MSE: 3.9015682970868317


In [43]:
import numpy as np

In [44]:
y_pred = regressor.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[86.02 85.  ]
 [86.71 90.  ]
 [87.75 89.  ]
 ...
 [90.49 87.  ]
 [92.42 94.  ]
 [89.95 88.  ]]


# PREDICT WITHOUT Pipline

In [20]:
wine_data=df

In [9]:
wine_data.columns

Index(['country', 'description', 'designation', 'points', 'price', 'province',
       'region_1', 'taster_name', 'title', 'variety', 'winery',
       'taster_weight_cat', 'clean_text', 'extracted_words',
       'extracted_digits'],
      dtype='object')

In [16]:
import numpy as np

In [43]:
X.columns

Index(['country', 'designation', 'price', 'province', 'region_1',
       'taster_name', 'variety', 'winery', 'taster_weight_cat', 'clean_text',
       'extracted_digits'],
      dtype='object')

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 1500)
X_count = cv.fit_transform(wine_data['clean_text']).toarray()
y = wine_data['points'].values

In [11]:
print(X_count.shape)

(129964, 1500)


In [49]:
wine_data.columns

Index(['country', 'description', 'designation', 'points', 'price', 'province',
       'region_1', 'taster_name', 'title', 'variety', 'winery',
       'taster_weight_cat', 'clean_text', 'extracted_words',
       'extracted_digits'],
      dtype='object')

In [56]:
enc_cols = wine_data.drop(['description', 'designation', 'points', 'price',  'title',  'clean_text'  ], axis=1)
dummies = pd.get_dummies(enc_cols, prefix=['country', 'province', 'region_1', 'variety','taster_name','taster_weight_cat','extracted_digits','winery','extracted_words'])

# combined with log transformed 'price'
X_encoded = pd.concat([np.log(wine_data['price']), dummies], axis = 1)
X_encoded.shape

(129964, 20844)

In [59]:
X_count_df = pd.DataFrame(X_count)
X = pd.concat([X_encoded, X_count_df], axis = 1)
X_train_count, X_test_count, y_train_count, y_test_count = train_test_split(X, y, test_size = 0.25, random_state = 12)

In [68]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction.text import CountVectorizer

# Drop unnecessary columns
enc_cols = wine_data.drop(['description', 'designation', 'points', 'price', 'title', 'clean_text','taster_name', 
                           'taster_weight_cat', 'extracted_digits', 'winery', 'extracted_words'], axis=1)

# Perform one-hot encoding
dummies = pd.get_dummies(enc_cols, prefix=['country', 'province', 'region_1', 'variety' ])

# Combine with log-transformed 'price'
X_encoded = pd.concat([np.log(wine_data['price']), dummies], axis=1)

# Apply CountVectorizer
cv = CountVectorizer(max_features=1000)
X_count = cv.fit_transform(wine_data['clean_text']).toarray()
X_count_df = pd.DataFrame(X_count)

# Combine the encoded features with the CountVectorizer features
X = pd.concat([X_encoded, X_count_df], axis=1)
X.columns = X.columns.astype(str)  # Convert column names to strings

# Split the data into train and test sets
X_train_count, X_test_count, y_train_count, y_test_count = train_test_split(X, y, test_size=0.25, random_state=12)

# Train the model
rf = RandomForestRegressor()
rf.fit(X_train_count, y_train_count)


RandomForestRegressor()

In [69]:
# test the model
y_pred_rf_count = rf.predict(X_test_count)

from sklearn import metrics
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test_count, y_pred_rf_count))  
print('Mean Squared Error:', metrics.mean_squared_error(y_test_count, y_pred_rf_count))  
print('Root Mean Squared Error:', np.sqrt(metrics.mean_squared_error(y_test_count, y_pred_rf_count)))

Mean Absolute Error: 1.3351813381288005
Mean Squared Error: 3.08882312023675
Root Mean Squared Error: 1.7575047994918107


# RandomForestRegressor

In [126]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import category_encoders as ce

# Load the dataset (assuming you have a CSV file named 'wine_data.csv')
wine_data = df

# Separate the features and target variable
X = wine_data.drop(['title', 'points', 'description', 'extracted_words', 'extracted_digits'], axis=1)
y = wine_data['points']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Define the column transformer
numeric_features = X.select_dtypes(include=['float64', 'int64']).columns
target_encoding_features = ['province', 'region_1', 'variety', 'designation', 'winery']
onehot_encoding_features = ['taster_weight_cat', 'taster_name', 'country']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('target_enc', ce.TargetEncoder(), target_encoding_features),
        ('onehot_enc', OneHotEncoder(handle_unknown='ignore'), onehot_encoding_features),
    ])

# Define the pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(n_estimators=100, max_depth=None, min_samples_split=2))
])

# Train the model
pipeline.fit(X_train, y_train)

# Evaluate the model on the test set
y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Test MSE:", mse)


Test MSE: 4.887396920951564


In [128]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import category_encoders as ce

# Load the dataset (assuming you have a CSV file named 'wine_data.csv')
wine_data = df

# Separate the features and target variable
X = wine_data.drop(['title', 'points', 'description', 'extracted_words', 'extracted_digits'], axis=1)
y = wine_data['points']

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Define the column transformer
numeric_features = X.select_dtypes(include=['float64', 'int64']).columns
target_encoding_features = ['province', 'region_1', 'variety', 'designation', 'winery']
onehot_encoding_features = ['taster_weight_cat', 'taster_name', 'country']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('target_enc', ce.TargetEncoder(), target_encoding_features),
        ('onehot_enc', OneHotEncoder(handle_unknown='ignore'), onehot_encoding_features),
    ])

# Define the pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor())
])

# Define the parameter grid for GridSearchCV
param_grid = {
    'model__n_estimators': [100, 200, 300],
    'model__max_depth': [None, 5, 10],
    'model__min_samples_split': [2, 5, 10]
}

# Perform GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Get the best model and its MSE
best_model = grid_search.best_estimator_
best_mse = -grid_search.best_score_

print("Best Model:")
print(best_model)
print("Best MSE:", best_mse)

# Evaluate the model on the test set
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("Test MSE:", mse)


KeyboardInterrupt: 